In [1]:
library(tidyverse)
library(repr)
library(digest)
library(gridExtra)
library(cowplot)
library(dplyr)
library(tidymodels)
library(GGally)
library(splines)

Warning message:
“package ‘tidyverse’ was built under R version 4.1.2”
── Attaching packages ─────────────────────────────────────── tidyverse 1.3.2 ──
✔ ggplot2 3.4.0      ✔ purrr   0.3.4 
✔ tibble  3.1.8      ✔ dplyr   1.0.10
✔ tidyr   1.2.0      ✔ stringr 1.4.0 
✔ readr   2.1.3      ✔ forcats 0.5.1 
Warning message:
“package ‘ggplot2’ was built under R version 4.1.2”
Warning message:
“package ‘tibble’ was built under R version 4.1.2”
Warning message:
“package ‘tidyr’ was built under R version 4.1.2”
Warning message:
“package ‘readr’ was built under R version 4.1.2”
Warning message:
“package ‘dplyr’ was built under R version 4.1.2”
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
Warning message:
“package ‘repr’ was built under R version 4.1.2”

Attaching package: ‘gridExtra’


The following object is masked from ‘package:dplyr’:

    combine


Warning message:
“package ‘tidym

In [2]:
setwd("/Users/matthewgillies/FantasyPreds2023")
data <- read_csv("data/Pitcher_base-stats.csv")
head(data)

New names:
• `` -> `...14`
Rows: 1836 Columns: 14
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr  (2): last_name, first_name
dbl (11): player_id, year, p_formatted_ip, p_total_hits, p_strikeout, p_walk...
lgl  (1): ...14

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


last_name,first_name,player_id,year,p_formatted_ip,p_total_hits,p_strikeout,p_walk,p_earned_run,p_save,p_win,p_loss,p_hold,...14
<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<lgl>
Wainwright,Adam,425794,2022,191.2,192,143,54,79,0,11,12,0,NA
Greinke,Zack,425844,2022,137.0,157,73,27,56,0,4,9,0,NA
Verlander,Justin,434378,2022,175.0,116,185,29,34,0,18,4,0,NA
Sanchez,Anibal,434671,2022,69.1,55,48,33,33,0,4,6,0,NA
Jansen,Kenley,445276,2022,64.0,45,85,22,24,41,5,2,0,NA
Chavez,Jesse,445926,2022,69.1,71,74,20,29,0,4,3,10,NA


In [3]:
selected <- data %>% select(-"player_id", -"...14") 
head(selected)

last_name,first_name,year,p_formatted_ip,p_total_hits,p_strikeout,p_walk,p_earned_run,p_save,p_win,p_loss,p_hold
<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Wainwright,Adam,2022,191.2,192,143,54,79,0,11,12,0
Greinke,Zack,2022,137.0,157,73,27,56,0,4,9,0
Verlander,Justin,2022,175.0,116,185,29,34,0,18,4,0
Sanchez,Anibal,2022,69.1,55,48,33,33,0,4,6,0
Jansen,Kenley,2022,64.0,45,85,22,24,41,5,2,0
Chavez,Jesse,2022,69.1,71,74,20,29,0,4,3,10


In [4]:
names <- selected %>% unite("player", last_name:first_name, sep = ", ", remove = T) %>% 
rename("IP" = "p_formatted_ip", "H" = "p_total_hits", "K" = "p_strikeout", "BB" = "p_walk", "ER" = "p_earned_run", "S" = "p_save",  
           "W" = "p_win", "L" = "p_loss", "HOLD" = "p_hold")
head(names)

player,year,IP,H,K,BB,ER,S,W,L,HOLD
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
"Wainwright, Adam",2022,191.2,192,143,54,79,0,11,12,0
"Greinke, Zack",2022,137.0,157,73,27,56,0,4,9,0
"Verlander, Justin",2022,175.0,116,185,29,34,0,18,4,0
"Sanchez, Anibal",2022,69.1,55,48,33,33,0,4,6,0
"Jansen, Kenley",2022,64.0,45,85,22,24,41,5,2,0
"Chavez, Jesse",2022,69.1,71,74,20,29,0,4,3,10


In [11]:
## Using scoring: H = -1, K = 1, BB = -1 ,ER = -2, IP = 3
## Disincluding all team related stats (Holds, Saves, Wins, Losses)
fpoints <- names %>% mutate(Fpoints = K - H - BB - 2*ER + 3*IP)
head(fpoints)

player,year,IP,H,K,BB,ER,S,W,L,HOLD,Fpoints
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
"Wainwright, Adam",2022,191.2,192,143,54,79,0,11,12,0,312.6
"Greinke, Zack",2022,137.0,157,73,27,56,0,4,9,0,188.0
"Verlander, Justin",2022,175.0,116,185,29,34,0,18,4,0,497.0
"Sanchez, Anibal",2022,69.1,55,48,33,33,0,4,6,0,101.3
"Jansen, Kenley",2022,64.0,45,85,22,24,41,5,2,0,162.0
"Chavez, Jesse",2022,69.1,71,74,20,29,0,4,3,10,132.3


In [12]:
fpointsperIP = fpoints %>% mutate(Fpointsper = Fpoints/IP)
fpointsperIP %>% filter(year == 2022)

player,year,IP,H,K,BB,ER,S,W,L,HOLD,Fpoints,Fpointsper
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
"Wainwright, Adam",2022,191.2,192,143,54,79,0,11,12,0,312.6,1.6349372
"Greinke, Zack",2022,137.0,157,73,27,56,0,4,9,0,188.0,1.3722628
"Verlander, Justin",2022,175.0,116,185,29,34,0,18,4,0,497.0,2.8400000
"Sanchez, Anibal",2022,69.1,55,48,33,33,0,4,6,0,101.3,1.4659913
"Jansen, Kenley",2022,64.0,45,85,22,24,41,5,2,0,162.0,2.5312500
"Chavez, Jesse",2022,69.1,71,74,20,29,0,4,3,10,132.3,1.9146165
"Detwiler, Ross",2022,26.1,31,28,10,13,1,0,2,1,39.3,1.5057471
"Kluber, Corey",2022,164.0,178,139,21,79,0,10,10,0,274.0,1.6707317
"Hill, Rich",2022,124.1,125,109,37,59,0,8,7,0,201.3,1.6220790


In [13]:
write_csv(fpointsperIP, "data/PitchFpointsper.csv")